In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = "2"
import re
import sys
from collections import Counter
from itertools import islice
from urllib.parse import urlparse, urlsplit, parse_qs, parse_qsl

import numpy as np
import parsel
from sklearn_crfsuite.metrics import flat_classification_report, sequence_accuracy_score

sys.path.insert(0, '..')
from autopager.storage import Storage
from autopager.htmlutils import (get_link_text, get_text_around_selector_list,
                                 get_link_href, get_selector_root)
from autopager.utils import (
    get_domain, normalize_whitespaces, normalize, ngrams, tokenize, ngrams_wb, replace_digits
)
from autopager.model import _num_tokens_feature, _elem_attr
from autopager import AUTOPAGER_LIMITS
from autopager.parserutils import (TagParser, MyHTMLParser, draw_scaled_page, position_check, compare_tag, get_first_tag)
parser = MyHTMLParser()
tagParser = TagParser()

In [2]:
import pandas as pd

In [3]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.sequence import pad_sequences
import random
tf.random.set_seed(0)
random.seed(0)
np.random.seed(0)

In [4]:
from tensorflow_addons.layers.crf import CRF

In [5]:
from tensorflow.keras.layers import (Dense, Input, Bidirectional, LSTM, Embedding, Masking, Concatenate,
                                    AveragePooling2D, MaxPooling2D, Reshape, Attention, GlobalAveragePooling1D
                                    , Activation, Conv1D, Conv2D, Flatten, Dropout)

In [6]:
tf.__version__

'2.3.0'

In [7]:
from ipywidgets import IntProgress
from IPython.display import display


In [8]:
storage = Storage()

Current test file:  ['en', 'zh', 'ko', 'ja', 'de', 'ru']


In [9]:
gpus = tf.config.experimental.list_physical_devices('GPU')

In [10]:
gpus

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [11]:
for device in gpus:
    tf.config.experimental.set_memory_growth(device, True)

In [12]:
%%time
urls = [rec['Page URL'] for rec in storage.iter_records(language='en',contain_button = True, file_type='T')]
X_raw, y, page_positions = storage.get_Xy(language='en',contain_button = True,  contain_position=True,file_type='T', scaled_page='normal')
print("pages: {}  domains: {}".format(len(urls), len({get_domain(url) for url in urls})))

Contain position: True
Finish: Get Page 1 (Encoding: UTF-8)records ... (len: 303)
Finish: Get Page 2 (Encoding: UTF-8)records ... (len: 243)
Finish: Get Page 3 (Encoding: UTF-8)records ... (len: 119)
Finish: Get Page 4 (Encoding: UTF-8)records ... (len: 944)
Finish: Get Page 5 (Encoding: UTF-8)records ... (len: 93)
Finish: Get Page 6 (Encoding: UTF-8)records ... (len: 994)
Finish: Get Page 7 (Encoding: UTF-8)records ... (len: 1014)
Finish: Get Page 8 (Encoding: UTF-8)records ... (len: 7)
Finish: Get Page 21 (Encoding: UTF-8)records ... (len: 158)
Finish: Get Page 22 (Encoding: UTF-8)records ... (len: 171)
Finish: Get Page 23 (Encoding: UTF-8)records ... (len: 181)
Finish: Get Page 24 (Encoding: UTF-8)records ... (len: 10)
Finish: Get Page 25 (Encoding: UTF-8)records ... (len: 165)
Finish: Get Page 26 (Encoding: UTF-8)records ... (len: 147)
Finish: Get Page 28 (Encoding: UTF-8)records ... (len: 268)
Finish: Get Page 33 (Encoding: UTF-8)records ... (len: 108)
Finish: Get Page 34 (Encodin

Finish: Get Page 267 (Encoding: UTF-8)records ... (len: 131)
Finish: Get Page 284 (Encoding: cp1252)records ... (len: 130)
Finish: Get Page 287 (Encoding: UTF-8)records ... (len: 82)
Finish: Get Page 288 (Encoding: UTF-8)records ... (len: 140)
Finish: Get Page 289 (Encoding: UTF-8)records ... (len: 44)
Finish: Get Page 293 (Encoding: UTF-8)records ... (len: 74)
Finish: Get Page 294 (Encoding: UTF-8)records ... (len: 63)
Finish: Get Page 295 (Encoding: UTF-8)records ... (len: 65)
Finish: Get Page 296 (Encoding: UTF-8)records ... (len: 20)
Finish: Get Page 299 (Encoding: UTF-8)records ... (len: 361)
Finish: Get Page 300 (Encoding: UTF-8)records ... (len: 361)
Finish: Get Page 301 (Encoding: UTF-8)records ... (len: 364)
Finish: Get Page 302 (Encoding: UTF-8)records ... (len: 170)
Finish: Get Page 303 (Encoding: UTF-8)records ... (len: 154)
Finish: Get Page 304 (Encoding: cp1252)records ... (len: 117)
Finish: Get Page 305 (Encoding: UTF-8)records ... (len: 1987)
Finish: Get Page 312 (Encod

In [13]:
max_page_seq = 512

In [14]:
from tensorflow.keras import Model
from tensorflow.data import Dataset

In [15]:
import copy

## Slice data into chunks

In [16]:
def filter_empty(x, y):
    res_x = [page for page in x if len(x)!= 0]
    res_y = [page for page in y if len(y)!= 0]
    return x, y

In [17]:
chunks_x, chunks_y, chunk_positions = X_raw, y, page_positions

In [175]:
chunks_x, chunks_y = filter_empty(chunks_x, chunks_y)

## Load Pre-trained Fastext model

In [14]:
from FastTextModel import FastTextModel

1 Physical GPUs, 1 Logical GPU


In [12]:
ft = FastTextModel()

Current dimension:  100


In [13]:
from numpy import dot
from numpy.linalg import norm

def cos_sim(a,b):
    ans = dot(a, b)/(norm(a)*norm(b))
    return ans

In [14]:
cos_sim(ft.getWordVector("last"), ft.getWordVector("next"))

0.80560714

## Load Pre-trained Laser embedding

In [18]:
from LaserSentenceModel import LaserSentenceModel

In [19]:
laser = LaserSentenceModel()

In [20]:
laser.getSentenceVector('hello').shape

(1024,)

In [21]:
def textToMultiVector(text):
    ngram_next = _as_list(ngrams_wb(replace_digits(text), 2, 5),AUTOPAGER_LIMITS.max_text_features)
    return np.average(laser.getSentenceVector(ngram_next), axis = 0)

In [21]:
cos_sim(laser.getSentenceVector('下一頁'), laser.getSentenceVector('next page'))

0.85246056

# Load Pre-trained Bert embedding

In [11]:
from BertModel import BertModel

1 Physical GPUs, 1 Logical GPU


In [12]:
bert_short_model = BertModel(128)

In [13]:
bert_long_model = BertModel(512)

In [14]:
pbert = bert_short_model

# Feature extraction

In [22]:
def parseAttribute(html):
    close_index = html.find('>')
    open_text = html[:close_index]
    open_text = open_text.replace('<a ','')
    open_text = open_text.replace('<button','')
    return normalize(open_text)

In [23]:
test_tags = X_raw[0].extract()

In [24]:
for tag in test_tags:
    text = parseAttribute(tag)
    print(normalize(text))

href="https://www.oneplus.com"
href="https://www.oneplus.com/8t"
href="https://www.oneplus.com/store"
href="https://www.oneplus.com/brand"
href="https://www.oneplus.com/support"
class="active" href="/"
href="https://www.oneplus.com/retail"
class="btn-collapsed js-pull-off-canvas" href="javascript:;" data-target=".js-off-canvas.js-menu"
href="//www.oneplus.com"
href="javascript:;" class="js-exit-off-canvas" style="font-size: 18px; float: right; padding: 0 20px; line-height: 3.5;"
href="//www.oneplus.com/8t"
href="//www.oneplus.com/store"
href="//www.oneplus.com/brand"
href="//www.oneplus.com/support"
href="/"
href="//www.oneplus.com/retail"
href="https://facebook.com/oneplusofficial" rel="nofollow"
href="https://twitter.com/oneplus" rel="nofollow"
href="https://instagram.com/oneplustech" rel="nofollow"
href="//forums.oneplus.com/help/terms"
href="//www.oneplus.com/privacy-and-legal"
href="javascript:%24('#login').submit();" id="nav_user" class="user-image"
href="/"
 id="showsearchbar" c

In [25]:
%%time
# XXX: these functions should be copy-pasted from autopager/model.py

def _as_list(generator, limit=None):
    """
    >>> _as_list(ngrams_wb("text", 2, 2), 0)
    []
    >>> _as_list(ngrams_wb("text", 2, 2), 2)
    ['te', 'ex']
    >>> _as_list(ngrams_wb("text", 2, 2))
    ['te', 'ex', 'xt']
    """
    return list(generator if limit is None else islice(generator, 0, limit))

def feat_to_tokens(feat, tokenizer):
    if type(feat) == type([]):
        feat = ' '.join(feat)
    tokens = tokenizer.tokenize(feat)
    return tokens

def num_token_feature_to_class(number):
    if number == '=0':
        return [1, 0, 0, 0]
    elif number == '=1':
        return [0, 1, 0, 0]
    elif number == '=2':
        return [0, 0, 1, 0]
    else:
        return [0, 0, 0, 1]

def link_to_features(link):
    text = normalize(get_link_text(link))
    href = get_link_href(link)
    if href is None:
        href = ""
    p = urlsplit(href)
    parent = link.xpath('..').extract()
    parent = get_first_tag(parser, parent[0])
    query_parsed = parse_qsl(p.query) #parse query string from path
    query_param_names = [k.lower() for k, v in query_parsed]
    query_param_names_ngrams = _as_list(ngrams_wb(
        " ".join([normalize(name) for name in query_param_names]), 3, 5, True
    ))
    attribute_text = parseAttribute(link.extract())
    # Classes of link itself and all its children.
    # It is common to have e.g. span elements with fontawesome
    # arrow icon classes inside <a> links.
    self_and_children_classes = ' '.join(link.xpath(".//@class").extract())
    parent_classes = ' '.join(link.xpath('../@class').extract())
    css_classes = normalize(parent_classes + ' ' + self_and_children_classes)
    
    token_feature = {
        'text-exact': replace_digits(text.strip()[:100].strip()),
#         'query': query_param_names,
        'query': query_param_names_ngrams,
        'parent-tag': parent,
#         'class': css_classes.split()[:AUTOPAGER_LIMITS.max_css_features],
        'class':_as_list(ngrams_wb(css_classes, 4, 5),
                          AUTOPAGER_LIMITS.max_css_features),
        'text': _as_list(ngrams_wb(replace_digits(text), 2, 5),
                         AUTOPAGER_LIMITS.max_text_features),
        'attribute_text': attribute_text,
    }
    tag_feature = {
        'isdigit': 1 if text.isdigit() is True else 0,
        'isalpha': 1 if text.isalpha() is True else 0,
        'has-href': 0 if href is "" else 1,
        'path-has-page': 1 if 'page' in p.path.lower() else 0,
        'path-has-pageXX': 1 if re.search(r'[/-](?:p|page\w?)/?\d+', p.path.lower()) is not None else 0,
        'path-has-number': 1 if any(part.isdigit() for part in p.path.split('/')) else 0,
        'href-has-year': 1 if re.search('20\d\d', href) is not None else 0,
        'class-has-disabled': 1 if 'disabled' in css_classes else 0,
#         'num-tokens': num_token_feature_to_class(_num_tokens_feature(text)),
    }
    non_token_feature = []
    for k,v in tag_feature.items():
        if type(v) == type([]):
            non_token_feature.extend(v)
        else:
            non_token_feature.append(v)

    return [token_feature, non_token_feature]


def page_to_features(xseq):
    feat_list = [link_to_features(a) for a in xseq]
    around = get_text_around_selector_list(xseq, max_length=15)
    
    return feat_list

CPU times: user 5 µs, sys: 1e+03 ns, total: 6 µs
Wall time: 9.3 µs


In [26]:
def get_token_tag_features_from_chunks(chunks):
    token_features = []
    tag_features = []
    for idx, page in enumerate(chunks):
        try:
            feat_list = page_to_features(page)
            token_features.append([node[0] for node in feat_list])
            tag_features.append(np.array([node[1] for node in feat_list]))
        except:
            raise Exception(f"Error occured on {idx}")
    return token_features, tag_features

In [27]:
def word_to_vector(word_list, word_vector_method = None):
    if word_vector_method is None:
        print("Need to specified a method.")
        return
    elif word_vector_method == 'FastText':
        if type(word_list) == type([]):
            if len(word_list) == 0:
                return np.zeros(ft.getModel().get_dimension())
            else:
                vectors_array = []
                for word in word_list:
                    vector = ft.getWordVector(word)
                    vectors_array.append(vector)
                mean_vector = np.mean(vectors_array, axis = 0)
                return mean_vector
        else:
            return ft.getWordVector(word_list)
    elif word_vector_method == 'Laser':
        return laser.getSentenceVector(word_list)

In [28]:
def pages_to_word_vector(ft, token_features):
    pages_vector = []
    for page in token_features:
        page_vectors = []
        for node in page:
            classes = word_to_vector(ft, node['class'])
            query = word_to_vector(ft, node['query'])
            p_tag = word_to_vector(ft, node['parent-tag'])
            full_vector = np.concatenate([classes, query, p_tag], axis = 0)
            page_vectors.append(full_vector)
        pages_vector.append(np.array(page_vectors))
    return pages_vector

In [29]:
token_features, tag_features = get_token_tag_features_from_chunks(chunks_x)
# train_tag_feature_token_list = extract_tokens_from_token_features(token_features)

In [30]:
token_features[10]

[{'text-exact': 'icontact',
  'query': [],
  'parent-tag': 'div',
  'class': ['cust',
   'usto',
   'stom',
   'tom-',
   'om-l',
   'm-lo',
   '-log',
   'logo',
   'custo',
   'ustom',
   'stom-',
   'tom-l',
   'om-lo',
   'm-log',
   '-logo',
   'hs-i',
   's-im',
   '-ima',
   'imag',
   'mage',
   'age-',
   'ge-w',
   'e-wi',
   '-wid',
   'widg',
   'idge',
   'dget',
   'hs-im',
   's-ima',
   '-imag',
   'image',
   'mage-',
   'age-w',
   'ge-wi',
   'e-wid',
   '-widg',
   'widge',
   'idget'],
  'text': ['ic',
   'co',
   'on',
   'nt',
   'ta',
   'ac',
   'ct',
   'ico',
   'con',
   'ont',
   'nta',
   'tac',
   'act',
   'icon',
   'cont',
   'onta',
   'ntac',
   'tact',
   'icont',
   'conta',
   'ontac',
   'ntact'],
  'attribute_text': 'href="https://www.icontact.com/"'},
 {'text-exact': 'features',
  'query': [],
  'parent-tag': 'li',
  'class': ['hs-m',
   's-me',
   '-men',
   'menu',
   'enu-',
   'nu-i',
   'u-it',
   '-ite',
   'item',
   'hs-me',
   's-men',

#### Attribute tokens

In [31]:
import itertools

In [32]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [33]:
fit_attr = []
attr_pages = [[node['attribute_text'].lower() for node in page] for page in token_features]
_ = [fit_attr.extend([node['attribute_text'].lower() for node in page]) for page in token_features]

In [35]:
tk = Tokenizer(num_words=None, char_level=True, oov_token='UNK')
tk.fit_on_texts(fit_attr)

In [36]:
# -----------------------Skip part start--------------------------
# construct a new vocabulary
alphabet = "abcdefghijklmnopqrstuvwxyz0123456789,;.!?:'\"/\\|_@#$%^&*~`+-=<>()[]{}"
char_dict = {}
for i, char in enumerate(alphabet):
    char_dict[char] = i + 1

# Use char_dict to replace the tk.word_index
tk.word_index = char_dict.copy()
# Add 'UNK' to the vocabulary
tk.word_index[tk.oov_token] = max(char_dict.values()) + 1
# -----------------------Skip part end----------------------------

In [37]:
# Convert string to index
train_sequences = [tk.texts_to_sequences(page) for page in attr_pages]

In [39]:
train_attr_data = [pad_sequences(train_page, maxlen=256, padding='post') for train_page in train_sequences]

In [40]:
train_attr_data = [np.array(train_page, dtype='float32') for train_page in train_attr_data]

In [43]:
vocab_size = len(tk.word_index)

69

In [44]:
embedding_weights = []
embedding_weights.append(np.zeros(vocab_size))

for char, i in tk.word_index.items():
    onehot = np.zeros(vocab_size)
    onehot[i-1] = 1
    embedding_weights.append(onehot)
embedding_weights = np.array(embedding_weights)

In [46]:
attr_embedding_size = 69
input_size = 256

In [47]:
attr_embedding_layer = Embedding(vocab_size+1, 
                                 attr_embedding_size,
                                 input_length=input_size,
                                 weights = [embedding_weights]
                                )

In [48]:
def get_test_attr(tk_train, test_token_features):
    test_attr_pages = [[node['attribute_text'].lower() for node in page] for page in test_token_features]
    test_sequences = [tk_train.texts_to_sequences(page) for page in test_attr_pages]
    test_attr_data = [pad_sequences(test_page, maxlen=256, padding='post') for test_page in test_sequences]
    test_attr_data = [np.array(test_page, dtype='float32') for test_page in test_attr_data]
    return test_attr_data

#### End Attribute Tokens

In [49]:
token_feature_list = list(token_features[0][0].keys())

In [50]:
def pages_to_word_vector_from_keylist(word_vector_method, token_features, word_to_vec_list = token_feature_list):
    print(f"Transform key {word_to_vec_list} to word_vector ... ")
    pages_vector = []
    p = IntProgress(max=len(token_features))
    p.description = '(Init)'
    p.value = 0
    display(p)
    for idx, page in enumerate(token_features):
        p.description = f"Task: {idx+1}"
        p.value = idx+1
        page_vectors = []
        for node in page:
            full_vector_list = []
            for k,v in node.items():
                if k in word_to_vec_list:
                    full_vector_list.append(word_to_vector(v, word_vector_method))
            full_vector = np.concatenate(full_vector_list, axis=0)
            page_vectors.append(full_vector)
        pages_vector.append(np.array(page_vectors))
    p.description = '(Done)'
    return pages_vector

In [51]:
def page_to_bert_embeddings(token_features, tokenizer):
    text_first_segs = []
    for page in token_features:
        page_one_features = []
        for node in page:
            text_exact = tokenizer.tokenize(node["text-exact"])
            page_one_features.append(["[CLS]"]+text_exact+["[SEP]"])
        text_first_segs.append(page_one_features)
    print("Start encode first seg embeddings")
    first_emb = pbert.page_list_to_bert_embedding_list(text_first_segs, Token=True)
    return first_emb

In [52]:
top_parent_tags = {}
for page in token_features:
    for node in page:
        p_tag = node['parent-tag']
        if p_tag not in top_parent_tags:
            top_parent_tags[p_tag] = 1
        else:
            top_parent_tags[p_tag] += 1
sorted_parent_tags = sorted(top_parent_tags.items(),key=lambda x:x[1],reverse=True)

In [53]:
data_map_for_ptag = sorted_parent_tags[:30]

In [54]:
def sparse_representation_with_map(tag, data_map = data_map_for_ptag):
    rt_vec = [0] * len(data_map)
    for idx, map_tag in enumerate(data_map):
        if tag == map_tag[0]:
            rt_vec[idx] = 1
            break
    return rt_vec

In [55]:
def get_ptags_vector(token_features):
    pages_ptag = []
    for page in token_features:
        ptag_page = []
        for node in page:
            p_tag = node['parent-tag']
            ptag_page.append(sparse_representation_with_map(p_tag))
        pages_ptag.append(np.array(ptag_page))
    return pages_ptag

In [56]:
ptags_vector = get_ptags_vector(token_features)

In [57]:
from collections import OrderedDict

In [58]:
class TagTokenizer:
    def __init__(self, myDict = None):
        rt_dict = {}
        rt_dict['[PAD]'] = 0
        rt_dict['[UNK]'] = 1
        i = 2
        if myDict is not None:
            for k,v in myDict.items():
                rt_dict[k] = i
                i+=1
        self.map = rt_dict
        
    def tokenize(self, word):
        if type(word) == type([]):
            token_list = []
            for _word in word:
                if _word not in self.map:
                    token_list.append(self.map['[UNK]'])
                else:
                    token_list.append(self.map[_word])
            return token_list
        else:
            if word not in self.map:
                return self.map['[UNK]']
            else:
                return self.map[word]
    def get_size(self):
        return len(self.map)

In [59]:
top_thousand_class = {}
top_thousand_query = {}
text_map = {}
for page in token_features:
    for node in page:
        for _class in node['class']:
            if _class in top_thousand_class:
                top_thousand_class[_class]+=1
            else:
                top_thousand_class[_class]=1
        for _query in node['query']:
            if _query in top_thousand_query:
                top_thousand_query[_query]+=1
            else:
                top_thousand_query[_query]=1
        for _text in node['text']:
            if _text not in text_map:
                text_map[_text] = 1

class_tokenizer = TagTokenizer(top_thousand_class)
query_tokenizer = TagTokenizer(top_thousand_query)
text_tokenizer = TagTokenizer(text_map)

In [60]:
print(class_tokenizer.get_size())
print(query_tokenizer.get_size())

21027
2896


In [61]:
# Use ft to encode all token_features
ft_full_tokens_emb = pages_to_word_vector_from_keylist('Laser', token_features, ['text-exact'])

Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=164)

In [62]:
ft_full_tokens_emb[0].shape

(303, 1024)

In [63]:
train_tag_info_list = tag_features #features which only have tag true/false information

## Padding to fixed size

In [64]:
def prepare_input_ids(page_tokens, max_len):
    pages_class = []
    pages_query = []
    pages_text = []
#     print(len(page_tokens))
    for page in page_tokens:
        class_page = []
        query_page = []
        text_page = []
        for node in page:
            #class
            class_ids = class_tokenizer.tokenize(node['class'])
            class_ids = class_ids + [0] * (max_len-len(class_ids))
            class_page.append(class_ids[:max_len])
            #query
            query_ids = query_tokenizer.tokenize(node['query'])
            query_ids = query_ids + [0] * (max_len-len(query_ids))
            query_page.append(query_ids[:max_len])
            #text
            text_ids = text_tokenizer.tokenize(node['text'])
            text_ids = text_ids + [0] * (max_len-len(text_ids))
            text_page.append(text_ids[:max_len])
        pages_class.append(np.array(class_page))
        pages_query.append(np.array(query_page))
        pages_text.append(np.array(text_page))
    return pages_class, pages_query, pages_text

In [65]:
max_len = 256

In [66]:
pages_class, pages_query, pages_text = prepare_input_ids(token_features, max_len)

In [67]:
train_attr_x = ft_full_tokens_emb

In [68]:
train_ptag = ptags_vector

In [69]:
train_tag_x = tag_features

In [70]:
train_tag_x = [ np.concatenate([tag_info,ptags], axis = 1) if len(tag_info)!=0 else np.array([]) for tag_info, ptags in zip(train_tag_x, train_ptag)]

In [124]:
train_composite_with_token = [train_attr_x, train_attr_data, pages_class, pages_query, train_tag_x]

In [72]:
train_composite_with_token = [train_attr_x, train_attr_data, train_tag_x]

In [73]:
labels = ["O", "PREV", "PAGE", "NEXT"]
tag2idx = { label:idx for idx,label in enumerate(labels)}
idx2tag = { idx:label for idx,label in enumerate(labels)}
num_tags = len(labels)

In [74]:
train_y = [np.array([tag2idx.get(l) for l in lab]) for lab in chunks_y]

In [125]:
for inputs in train_composite_with_token:
    print(inputs[0].shape)

(303, 1024)
(303, 256)
(303, 256)
(303, 256)
(303, 38)


# custom Training

In [77]:
conv_layers = [[256, 3, 3],
               [256, 3, 3],
               [256, 5, -1],
               [256, 5, -1],
               [256, 5, 3]
              ]
dropout_p = 0.25
fully_connected_layers = [512, 512]
optimizer = keras.optimizers.Adam()

In [149]:
ft_shape = (None, 1024)
tag_info_shape = (None, 38)
tag_emb_shape = (None, 256)
pool_size = (256, 1)
HIDDEN_UNITS = 300
NUM_CLASS = num_tags

input_ft_embedding = Input(shape=(ft_shape), name="input_ft_embeddings")
input_tag_information = Input(shape=(tag_info_shape), name="input_tag_information")
input_attribute = Input(shape=(tag_emb_shape), name="input_attribute")

# Char-CNN Attribute start
attr_emb = attr_embedding_layer(input_attribute)
# attr_emb = AveragePooling2D(pool_size, data_format = 'channels_first')(attr_emb)
# attr_emb = Reshape((-1, 69), name="query_emb_out")(attr_emb)

In [150]:
attr_emb.shape

TensorShape([None, None, 256, 69])

In [155]:
attr_embedding_layer

In [159]:
print(attr_embedding_size)

69


In [160]:
print(input_size)

256


In [161]:
page_size = None
ft_shape = (page_size, 1024)
tag_info_shape = (page_size, 38)
tag_emb_shape = (page_size, 256)
HIDDEN_UNITS = 300
NUM_CLASS = num_tags

input_ft_embedding = Input(shape=(ft_shape), name="input_ft_embeddings")
input_tag_information = Input(shape=(tag_info_shape), name="input_tag_information")
input_attribute = Input(shape=(tag_emb_shape), name="input_attribute")
input_class = Input(shape=(tag_emb_shape), name="input_class")
input_query = Input(shape=(tag_emb_shape), name="input_query")
print(input_attribute.shape)
# Char-CNN Attribute start
attr_emb = Embedding(vocab_size+1,attr_embedding_size,input_length=input_size,weights = [embedding_weights])(input_attribute)
# attr_emb = AveragePooling2D(pool_size, data_format = 'channels_first')(attr_emb)
# attr_emb = Reshape((-1, 69), name="attribute_emb_out")(attr_emb)
# attr_emb = Dense(units = 256, activation = 'relu')(attr_emb)
# attr_emb = Dense(units = 128, activation = 'relu')(attr_emb)
print(attr_emb.shape)
for filter_num, filter_size, pooling_size in conv_layers:
    attr_emb = Conv1D(filter_num, filter_size, input_shape = attr_emb.shape[2:])(attr_emb)
    attr_emb = Activation('relu')(attr_emb)
    print(attr_emb.shape)
    if pooling_size != -1:
        attr_emb = MaxPooling2D(pool_size=(1, pooling_size))(attr_emb)  # Final shape=(None, 34, 256)
attr_shape = attr_emb.get_shape().as_list()
if page_size is None:
    attr_emb = Reshape((-1, attr_shape[2] * attr_shape[3]))(attr_emb)
else:
    attr_emb = Reshape((page_size, attr_shape[2] * attr_shape[3]))(attr_emb)
for dense_size in fully_connected_layers:
    attr_emb = Dense(dense_size, activation='relu')(attr_emb)  # dense_size == 1024
    attr_out = Dropout(dropout_p)(attr_emb)

# Char-CNN Attribute end

ft_FFN = Dense(units = 512, activation = 'relu', name="ft_FFN_01")(input_ft_embedding)
ft_FFN = Dense(units = 256, activation = 'relu', name="ft_FFN_02")(ft_FFN)
ft_FFN = Dense(units = 128, activation = 'relu', name="ft_FFN_out")(ft_FFN)



merged = Concatenate()([ft_FFN, attr_emb, input_tag_information])
#     model = Bidirectional(LSTM(units = HIDDEN_UNITS//2, return_sequences=True))(merged)
model = LSTM(units = HIDDEN_UNITS, return_sequences=True)(merged)

crf=CRF(NUM_CLASS, name='crf_layer')
out =crf(model)
model = Model([input_ft_embedding, input_attribute, input_class, input_query, input_tag_information], out)
loss_fn = crf.get_loss

(None, None, 256)
(None, None, 256, 69)
(None, None, 254, 256)
(None, None, 82, 256)
(None, None, 23, 256)
(None, None, 19, 256)
(None, None, 15, 256)


In [80]:
#For custom embedding
def get_char_cnn_emb_model(use_crf = True):
    ft_shape = (None, 1024)
    tag_info_shape = (None, 38)
    tag_emb_shape = (None, 256)
    HIDDEN_UNITS = 300
    NUM_CLASS = num_tags

    input_ft_embedding = Input(shape=(ft_shape), name="input_ft_embeddings")
    input_tag_information = Input(shape=(tag_info_shape), name="input_tag_information")
    input_attribute = Input(shape=(tag_emb_shape), name="input_class")

    # Char-CNN Attribute start
    attr_emb = Embedding(vocab_size+1,attr_embedding_size,input_length=input_size,weights = [embedding_weights])(input_attribute)

    for filter_num, filter_size, pooling_size in conv_layers:
        attr_emb = Conv2D(filter_num, filter_size)(attr_emb)
        attr_emb = Activation('relu')(attr_emb)
        if pooling_size != -1:
            attr_emb = MaxPooling2D(pool_size=(1, pooling_size))(attr_emb)  # Final shape=(None, 34, 256)
    attr_shape = attr_emb.get_shape().as_list()
    attr_emb = Reshape((-1, attr_shape[2] * attr_shape[3]))(attr_emb)
    for dense_size in fully_connected_layers:
        attr_emb = Dense(dense_size, activation='relu')(attr_emb)  # dense_size == 1024
        attr_out = Dropout(dropout_p)(attr_emb)

    # Char-CNN Attribute end

    ft_FFN = Dense(units = 512, activation = 'relu', name="ft_FFN_01")(input_ft_embedding)
    ft_FFN = Dense(units = 256, activation = 'relu', name="ft_FFN_02")(ft_FFN)
    ft_FFN = Dense(units = 128, activation = 'relu', name="ft_FFN_out")(ft_FFN)



    merged = Concatenate()([ft_FFN, attr_out, input_tag_information])
    #     model = Bidirectional(LSTM(units = HIDDEN_UNITS//2, return_sequences=True))(merged)
    model = LSTM(units = HIDDEN_UNITS, return_sequences=True)(merged)

    crf=CRF(NUM_CLASS, name='crf_layer')
    out =crf(model)
    model = Model([input_ft_embedding, input_attribute, input_tag_information], out)

    loss_fn = crf.get_loss
    return model, loss_fn

In [151]:
test, _ = get_char_cnn_emb_model()

In [152]:
test.summary()

Model: "functional_24"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_class (InputLayer)        [(None, None, 256)]  0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             4830        input_class[0][0]                
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, 254, 25 159232      embedding[15][0]                 
__________________________________________________________________________________________________
max_pooling2d_36 (MaxPooling2D) (None, None, 84, 256 0           conv2d[0][0]                     
______________________________________________________________________________________

In [164]:
#For custom embedding

def get_char_cnn_attr_emb_model(use_crf = True, embedding_size = 30):
    ft_shape = (None, 1024)
    tag_info_shape = (None, 38)
    tag_emb_shape = (None, 256)
    HIDDEN_UNITS = 300
    NUM_CLASS = num_tags
    embbed_output_shape = embedding_size
    page_embbed_shape = (-1, embbed_output_shape)

    input_ft_embedding = Input(shape=(ft_shape), name="input_ft_embeddings")
    input_tag_information = Input(shape=(tag_info_shape), name="input_tag_information")
    input_attribute = Input(shape=(tag_emb_shape), name="input_attr")
    input_class = Input(shape=(tag_emb_shape), name="input_class")
    input_query = Input(shape=(tag_emb_shape), name="input_query")

    # Char-CNN Attribute start
    
    
    class_emb = Embedding(input_dim = class_tokenizer.get_size(), output_dim = embbed_output_shape, input_length=max_page_seq, mask_zero = True)(input_class)
    class_emb = AveragePooling2D((256, 1), data_format = 'channels_first')(class_emb)
    class_emb = Reshape(page_embbed_shape, name="class_emb_out")(class_emb)
    ## input_query
    query_emb = Embedding(input_dim = query_tokenizer.get_size(), output_dim = embbed_output_shape, input_length=max_page_seq, mask_zero = True)(input_query)
    query_emb = AveragePooling2D((256, 1), data_format = 'channels_first')(query_emb)
    query_emb = Reshape(page_embbed_shape, name="query_emb_out")(query_emb)
    
    attr_emb = Embedding(vocab_size+1,attr_embedding_size,input_length=input_size,weights = [embedding_weights])(input_attribute)
    for filter_num, filter_size, pooling_size in conv_layers:
        attr_emb = Conv1D(filter_num, filter_size, input_shape = attr_emb.shape[2:])(attr_emb)
        attr_emb = Activation('relu')(attr_emb)
        if pooling_size != -1:
            attr_emb = MaxPooling2D(pool_size=(1, pooling_size))(attr_emb)  # Final shape=(None, 34, 256)
    attr_shape = attr_emb.get_shape().as_list()
    attr_emb = Reshape((-1, attr_shape[2] * attr_shape[3]))(attr_emb)
    attr_emb_merged = attr_emb
    attr_emb_merged = Concatenate()([attr_emb, class_emb, query_emb])
    for dense_size in fully_connected_layers:
        attr_emb_merged = Dense(dense_size, activation='relu')(attr_emb_merged)  # dense_size == 1024
    
    # Char-CNN Attribute end
    ft_FFN = Dense(units = 512, activation = 'relu', name="ft_FFN_01")(input_ft_embedding)
    ft_FFN = Dense(units = 256, activation = 'relu', name="ft_FFN_02")(ft_FFN)
    ft_FFN = Dense(units = 128, activation = 'relu', name="ft_FFN_out")(ft_FFN)

    merged = Concatenate()([ft_FFN, attr_emb_merged, input_tag_information])
    model = Bidirectional(LSTM(units = HIDDEN_UNITS//2, return_sequences=True))(merged)
#     model = LSTM(units = HIDDEN_UNITS, return_sequences=True)(merged)

    crf=CRF(NUM_CLASS, name='crf_layer')
    out =crf(model)
    model = Model([input_ft_embedding, input_attribute, input_class, input_query, input_tag_information], out)

    loss_fn = crf.get_loss
    return model, loss_fn

In [165]:
batch_size = 1

In [128]:
def list_to_dataSet(data, dataType):
    dataset = Dataset.from_generator(lambda: iter(data), dataType)
    return dataset
def zip_dataSet(data):
    data_tuple = tuple(data)
    dataset = Dataset.zip(data_tuple)
    return dataset
def describe_dataset(dataset):
    print(train_dataset.element_spec)

In [129]:
def composite_splite_to_train_val(composite_x, y, number):
    x_train = [ data[:-number] for data in composite_x]
    y_train = y[:-number]
    x_val = [ data[-number:] for data in composite_x]
    y_val = y[-number:]
    return x_train, y_train, x_val, y_val

In [130]:
def composite_cut_data(composite_x, y, percent):
    number = round(len(y) * percent)
    new_composite_x = [ data[:number] for data in composite_x]
    new_y = y[:number]

    return new_composite_x, new_y

In [131]:
x_train, y_train, x_val, y_val = composite_splite_to_train_val(train_composite_with_token, train_y, 20)

In [132]:
def data_list_to_dataset(x, y, isValidation = False, batch_size = 1):
    all_data = None
    for data in x:
        dataset = list_to_dataSet(data, tf.float32)
        if all_data == None:
            all_data = dataset
        else:
            all_data = Dataset.zip((all_data, dataset))
    y_ds = list_to_dataSet(y, tf.int32)
    final_set = Dataset.zip((all_data, y_ds))
    if not isValidation:
        final_set = final_set.shuffle(buffer_size=1024).batch(batch_size)
    else:
        final_set = final_set.batch(batch_size)
    return final_set

def composite_list_to_dataset(x, batch_size = 1):
    all_data = None
    for data in x:
        dataset = list_to_dataSet(data, tf.float32)
        if all_data == None:
            all_data = dataset
        else:
            all_data = Dataset.zip((all_data, dataset))
    return all_data.batch(batch_size)

In [133]:
train_dataset = data_list_to_dataset(x_train, y_train, isValidation=False)

In [134]:
val_dataset = data_list_to_dataset(x_val, y_val, isValidation=True)

In [135]:
from sklearn.metrics import classification_report
from collections import Counter
def calculate_pages_metric(y_true_pages, y_predict_pages):
    pages_f1 = []
    nexts_f1 = []
    avg_f1 = []
    for y_true, y_predict in zip(y_true_pages, y_predict_pages):
        if len(y_true) == 0:
            break
        report = classification_report(y_true, y_predict,output_dict=True)
#         print(report)
        PAGE = report['2']['f1-score']
        NEXT = report['3']['f1-score']
        pages_f1.append(PAGE)
        nexts_f1.append(NEXT)
        avg_f1.append((PAGE+NEXT)/2)
    return pages_f1, nexts_f1, avg_f1
def calculate_page_metric(y_true, y_predict):    
    report = classification_report(y_true, y_predict,labels=[0,2,3],output_dict=True)
    OTHER = report['0']['f1-score']
    PAGE = report['2']['f1-score']
    NEXT = report['3']['f1-score']
    if 2 in y_true and 3 in y_true:
        AVG = (PAGE+NEXT)/2
    elif 2 in y_true and 3 not in y_true:
        AVG = PAGE
    elif 2 not in y_true and 3 in y_true:
        AVG = NEXT
    else:
        AVG = OTHER
    return AVG

In [166]:
model, loss_fn = get_char_cnn_attr_emb_model()

In [167]:
model.summary()

Model: "functional_34"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_attr (InputLayer)         [(None, None, 256)]  0                                            
__________________________________________________________________________________________________
embedding_18 (Embedding)        (None, None, 256, 69 4830        input_attr[0][0]                 
__________________________________________________________________________________________________
conv1d_80 (Conv1D)              (None, None, 254, 25 53248       embedding_18[0][0]               
__________________________________________________________________________________________________
activation_80 (Activation)      (None, None, 254, 25 0           conv1d_80[0][0]                  
______________________________________________________________________________________

In [138]:
for (x,y) in train_dataset.take(1):
    print(x)
    print(y)
    break

((((<tf.Tensor: shape=(1, 131, 1024), dtype=float32, numpy=
array([[[ 6.4000086e-04,  1.8474861e-04,  9.5328875e-04, ...,
         -1.3147385e-04, -1.7696819e-03, -2.5204704e-03],
        [ 1.6496029e-02, -7.1148439e-05,  2.0087403e-03, ...,
          6.8351980e-03,  3.7382210e-03,  1.9063668e-02],
        [ 6.4000086e-04,  1.8474861e-04,  9.5328875e-04, ...,
         -1.3147385e-04, -1.7696819e-03, -2.5204704e-03],
        ...,
        [ 1.2916614e-03,  9.3595136e-04, -1.8469816e-03, ...,
          7.6044090e-03,  1.5522135e-04,  3.0489087e-03],
        [ 1.2916614e-03,  9.3595136e-04, -1.8469816e-03, ...,
          7.6044090e-03,  1.5522135e-04,  3.0489087e-03],
        [ 1.2916614e-03,  9.3595136e-04, -1.8469816e-03, ...,
          7.6044090e-03,  1.5522135e-04,  3.0489087e-03]]], dtype=float32)>, <tf.Tensor: shape=(1, 131, 256), dtype=float32, numpy=
array([[[ 9.,  4., 60., ...,  0.,  0.,  0.],
        [ 8., 18.,  5., ...,  0.,  0.,  0.],
        [ 3., 12.,  1., ...,  0.,  0.,  0.]

In [168]:
#Test for data predict
for (batch_x, batch_y) in train_dataset.take(1):
    batch_predict_y = model(batch_x).numpy()
    batch_true_y = batch_y.numpy()
    print(batch_true_y)
    print(calculate_page_metric(batch_true_y[0], batch_predict_y[0]))
    print(classification_report(batch_true_y[0], batch_predict_y[0]))
    break

[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
  0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]
0.4
              precision    recall  f1-score   support

           0       1.00      0.25      0.40       268
           1       0.00      0.00      0.00         0
           2       0.00      0.00      0.00         0
           3       0.00      0.00      0.00         0

    accuracy                           0.25       268
   macro avg       0.25      0.06      0.10       268
weighted avg       1.00      0.25      0.40       268



/home/sam/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [169]:
def train_on_epoch(epochs, model, optimizer, train_dataset, val_dataset, best_model_method = 'f1-score'):
    import time
    
    epochs = epochs
    best_weights = None
    best_f1_weights = None
    best = np.Inf
    best_loss_history = None
    best_f1 = 0
    best_f1_history = None
    avg_epoch_losses = []
    avg_epoch_f1s = []
    for epoch in range(epochs):
        print("\nStart of epoch %d" % (epoch,))
        start_time = time.time()

        # Iterate over the batches of the dataset.
        for step, (x_batch_train, y_batch_train) in enumerate(train_dataset):
            with tf.GradientTape() as tape:
                logits = model(x_batch_train, training=True)
                loss_value = loss_fn(y_batch_train, logits)
            grads = tape.gradient(loss_value, model.trainable_weights)
            optimizer.apply_gradients(zip(grads, model.trainable_weights))

            # Log every 50 batches.
#             if step % 50 == 0:
#                 print(
#                     "Training loss (for one batch) at step %d: %.4f"
#                     % (step, float(loss_value))
#                 )
#                 print("Seen so far: %d samples" % ((step + 1) * batch_size))


        # Run a validation loop at the end of each epoch.
        val_losses = []
        val_f1s = []
        for x_batch_val, y_batch_val in val_dataset:
            val_logits = model(x_batch_val, training=False)
            val_loss_value = loss_fn(y_batch_val, val_logits)
            val_avg_f1 = calculate_page_metric(y_batch_val.numpy()[0], val_logits.numpy()[0])
            val_losses.append(val_loss_value)
            val_f1s.append(val_avg_f1)
        average_val_loss = np.average(val_losses)
        average_val_f1 = np.average(val_f1s)
        avg_epoch_losses.append(average_val_loss)
        avg_epoch_f1s.append(average_val_f1)
        if average_val_loss < best:
            best_weights = model.get_weights()
            best = average_val_loss
            best_loss_history = [val_losses, val_f1s]
        if average_val_f1 > best_f1:
            best_f1_weights = model.get_weights()
            best_f1 = average_val_f1
            best_f1_history = [val_losses, val_f1s]
        print("Validation loss: %.4f" % (float(average_val_loss),))
        print("Validation F1: %.4f" % (float(average_val_f1),))
        print("Time taken: %.2fs" % (time.time() - start_time))
    print(f"Best loss: {best}, Best F1: {best_f1}")
    print(f"Training finish, load best weights. {best_model_method}")
    
    if best_model_method == 'loss':
        model.set_weights(best_weights)
    elif best_model_method == 'f1-score':
        model.set_weights(best_f1_weights)
    avg_epoch_result = {"epoch_losses": avg_epoch_losses, "epoch_f1s": avg_epoch_f1s}
    return model, avg_epoch_result

In [190]:
model, avg_epoch_result = train_on_epoch(25, model, optimizer, train_dataset, val_dataset)


Start of epoch 0
Validation loss: 19.0802
Validation F1: 0.5410
Time taken: 85.91s

Start of epoch 1
Validation loss: 17.6187
Validation F1: 0.5075
Time taken: 72.94s

Start of epoch 2
Validation loss: 26.8368
Validation F1: 0.3437
Time taken: 73.81s

Start of epoch 3
Validation loss: 16.7178
Validation F1: 0.8037
Time taken: 73.29s

Start of epoch 4
Validation loss: 24.8617
Validation F1: 0.7777
Time taken: 74.05s

Start of epoch 5
Validation loss: 17.9413
Validation F1: 0.8037
Time taken: 72.13s

Start of epoch 6
Validation loss: 22.7943
Validation F1: 0.8110
Time taken: 72.23s

Start of epoch 7
Validation loss: 21.8403
Validation F1: 0.8228
Time taken: 74.65s

Start of epoch 8
Validation loss: 26.8399
Validation F1: 0.8134
Time taken: 74.46s

Start of epoch 9
Validation loss: 24.9545
Validation F1: 0.8211
Time taken: 73.66s

Start of epoch 10
Validation loss: 25.9984
Validation F1: 0.7992
Time taken: 73.71s

Start of epoch 11
Validation loss: 24.8515
Validation F1: 0.8316
Time take

In [199]:
reports = evaluate_model(model)

Testing language:  en
pages: 49  domains: 34
Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=49)

Start predicting test data ...
Start evaluating test data ...
                 PAGE       NEXT   micro avg   macro avg  weighted avg
precision    0.879699   0.735294    0.850299    0.807497      0.852682
recall       0.928571   0.862069    0.916129    0.895320      0.916129
f1-score     0.903475   0.793651    0.881988    0.848563      0.882927
support    126.000000  29.000000  155.000000  155.000000    155.000000
Testing language:  de


/home/sam/DL/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['PAGE', 'NEXT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


pages: 20  domains: 7
Transform key ['text-exact'] to word_vector ... 


...

/home/sam/DL/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['PAGE', 'NEXT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


pages: 24  domains: 13
Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=24)

Start predicting test data ...
Start evaluating test data ...
                PAGE  NEXT  micro avg  macro avg  weighted avg
precision   0.584158   0.0   0.561905   0.292079      0.539223
recall      0.983333   0.0   0.907692   0.491667      0.907692
f1-score    0.732919   0.0   0.694118   0.366460      0.676541
support    60.000000   5.0  65.000000  65.000000     65.000000


/home/sam/DL/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['PAGE', 'NEXT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [174]:
pd.DataFrame(reports)

,en,de,ru,zh,ja,ko
PAGE,"{'precision': 0.6781609195402298, 'recall': 0....","{'precision': 0.764367816091954, 'recall': 0.9...","{'precision': 0.7906976744186046, 'recall': 0....","{'precision': 0.49742268041237114, 'recall': 0...","{'precision': 0.5052631578947369, 'recall': 0....","{'precision': 0.33519553072625696, 'recall': 1..."
NEXT,"{'precision': 0.45901639344262296, 'recall': 0...","{'precision': 0.4888888888888889, 'recall': 1....","{'precision': 1.0, 'recall': 1.0, 'f1-score': ...","{'precision': 0.5405405405405406, 'recall': 0....","{'precision': 0.5625, 'recall': 0.818181818181...","{'precision': 0.3125, 'recall': 1.0, 'f1-score..."
micro avg,"{'precision': 0.6212765957446809, 'recall': 0....","{'precision': 0.7077625570776256, 'recall': 0....","{'precision': 0.82, 'recall': 0.93181818181818...","{'precision': 0.5011764705882353, 'recall': 0....","{'precision': 0.5135135135135135, 'recall': 0....","{'precision': 0.3333333333333333, 'recall': 1...."
macro avg,"{'precision': 0.5685886564914264, 'recall': 0....","{'precision': 0.6266283524904215, 'recall': 0....","{'precision': 0.8953488372093024, 'recall': 0....","{'precision': 0.5189816104764559, 'recall': 0....","{'precision': 0.5338815789473684, 'recall': 0....","{'precision': 0.3238477653631285, 'recall': 1...."
weighted avg,"{'precision': 0.6371596856251937, 'recall': 0....","{'precision': 0.7267247328716595, 'recall': 0....","{'precision': 0.8239957716701902, 'recall': 0....","{'precision': 0.5008606493262451, 'recall': 0....","{'precision': 0.5157565789473685, 'recall': 0....","{'precision': 0.333449720670391, 'recall': 1.0..."


In [200]:
calculate_macro_avg(reports)

0.7685706291510725

In [170]:
best_model = None
best = 0
for iters in range(3):
    model, loss_fn = get_char_cnn_attr_emb_model()
    model, avg_epoch_result = train_on_epoch(25, model, optimizer, train_dataset, val_dataset)
    reports = evaluate_model(model)
    macro_f1 = calculate_macro_avg(reports)
    if macro_f1 > best:
        best_model = model.get_weights()
        best = macro_f1
    print(f"Iteration: {iters}, Macro F1: {macro_f1}")


Start of epoch 0
ERROR:tensorflow:==================================
Object was never used (type <class 'tensorflow.python.ops.tensor_array_ops.TensorArray'>):
If you want to mark it as used call its "mark_used()" method.
It was originally created here:
  File "/home/sam/DL/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py", line 2778, in while_loop
    return result  File "/home/sam/DL/lib/python3.6/site-packages/tensorflow/python/ops/control_flow_ops.py", line 2726, in <lambda>
    body = lambda i, lv: (i + 1, orig_body(*lv))  File "/home/sam/DL/lib/python3.6/site-packages/tensorflow/python/ops/functional_ops.py", line 656, in compute
    return (next_i, flat_a_out, tas)  File "/home/sam/DL/lib/python3.6/site-packages/tensorflow/python/ops/functional_ops.py", line 651, in <listcomp>
    tas = [ta.write(i, value) for (ta, value) in zip(tas, flat_a_out)]  File "/home/sam/DL/lib/python3.6/site-packages/tensorflow/python/util/tf_should_use.py", line 249, in wrapped
 

/home/sam/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation loss: 26.1245
Validation F1: 0.7514
Time taken: 77.95s

Start of epoch 1
Validation loss: 29.8757
Validation F1: 0.7079
Time taken: 78.06s

Start of epoch 2
Validation loss: 21.6204
Validation F1: 0.7447
Time taken: 80.03s

Start of epoch 3
Validation loss: 19.9772
Validation F1: 0.6501
Time taken: 79.57s

Start of epoch 4
Validation loss: 26.7768
Validation F1: 0.7751
Time taken: 76.78s

Start of epoch 5
Validation loss: 25.6591
Validation F1: 0.8088
Time taken: 78.73s

Start of epoch 6
Validation loss: 37.9545
Validation F1: 0.8873
Time taken: 78.79s

Start of epoch 7
Validation loss: 24.4895
Validation F1: 0.8111
Time taken: 79.60s

Start of epoch 8
Validation loss: 28.4088
Validation F1: 0.8254
Time taken: 78.79s

Start of epoch 9
Validation loss: 23.4112
Validation F1: 0.8063
Time taken: 79.28s

Start of epoch 10
Validation loss: 43.0422
Validation F1: 0.8269
Time taken: 78.39s

Start of epoch 11
Validation loss: 32.9948
Validation F1: 0.8700
Time taken: 76.86s

Start o

IntProgress(value=0, description='(Init)', max=49)

Start predicting test data ...
Start evaluating test data ...
                 PAGE       NEXT   micro avg   macro avg  weighted avg
precision    0.678161   0.459016    0.621277    0.568589      0.637160
recall       0.936508   0.965517    0.941935    0.951013      0.941935
f1-score     0.786667   0.622222    0.748718    0.704444      0.755900
support    126.000000  29.000000  155.000000  155.000000    155.000000
Testing language:  de


...

/home/sam/DL/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['PAGE', 'NEXT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)



Start of epoch 0


/home/sam/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation loss: 40.9903
Validation F1: 0.2999
Time taken: 77.48s

Start of epoch 1
Validation loss: 40.2714
Validation F1: 0.2999
Time taken: 77.64s

Start of epoch 2
Validation loss: 35.5608
Validation F1: 0.2999
Time taken: 77.39s

Start of epoch 3
Validation loss: 34.8320
Validation F1: 0.2999
Time taken: 80.20s

Start of epoch 4
Validation loss: 34.0868
Validation F1: 0.2999
Time taken: 79.02s

Start of epoch 5
Validation loss: 31.6334
Validation F1: 0.2999
Time taken: 78.99s

Start of epoch 6
Validation loss: 41.2002
Validation F1: 0.2999
Time taken: 78.87s

Start of epoch 7
Validation loss: 30.0784
Validation F1: 0.2999
Time taken: 79.60s

Start of epoch 8
Validation loss: 29.1931
Validation F1: 0.2999
Time taken: 78.41s

Start of epoch 9
Validation loss: 28.7037
Validation F1: 0.2999
Time taken: 77.54s

Start of epoch 10
Validation loss: 30.1072
Validation F1: 0.2999
Time taken: 78.34s

Start of epoch 11


KeyboardInterrupt: 

## Test on batch

In [143]:
def prepare_for_testing(test_X_raw, test_y_raw): #ft-bert -no chunks
    chunks_test_x, chunks_test_y = test_X_raw, test_y_raw
    chunks_test_x, chunks_test_y = filter_empty(chunks_test_x, chunks_test_y)
    test_token_features, test_tag_features = get_token_tag_features_from_chunks(chunks_test_x)
    
    test_ptags_vector = get_ptags_vector(test_token_features)
    test_ft_emb = pages_to_word_vector_from_keylist('Laser', test_token_features, ['text-exact'])
    test_attr = get_test_attr(tk, test_token_features)
    test_pages_class, test_pages_query, _ = prepare_input_ids(test_token_features, max_len)
    test_tag_info_list = test_tag_features

    ## X_test_input
    test_tag_x = [ np.concatenate([tag_info,ptags], axis = 1) if len(tag_info)!=0 else np.array([]) for tag_info, ptags in zip(test_tag_info_list, test_ptags_vector)]
    test_composite_input = [test_ft_emb, test_attr, test_pages_class, test_pages_query, test_tag_x]
    
    ## y_test_input
    y_test = [[tag2idx.get(l) for l in lab] for lab in chunks_test_y]
    y_test = [[idx2tag.get(lab) for lab in page] for page in y_test]
    y_test = np.asarray(y_test)
    
    return test_composite_input, y_test

In [144]:
def evaluate_from_batch(model, x, y, evaluate_labels, multiTask = False):
    print("Start predicting test data ...")
    test_page_dataset = composite_list_to_dataset(x)
    predicted_y = []
    for pageIdx, batch_x_test in enumerate(test_page_dataset):
        if len(y[pageIdx]) == 0:
            batch_predict_y = np.array([])
        else:
            if multiTask:
                batch_predict_y = model(batch_x_test)[0][0].numpy()
            else:
                batch_predict_y = model(batch_x_test)[0].numpy()
#         print(batch_predict_y.shape)
        if len(batch_predict_y.shape) != 1:
            tmp = list()
            for lab in batch_predict_y:
                lab = lab.tolist()
                tmp.append(lab.index(max(lab)))
            batch_predict_y = tmp
        predicted_y.append(batch_predict_y)
    print("Start evaluating test data ...")
    predict_y = np.asarray([[idx2tag.get(lab) for lab in page] for page in predicted_y])
    report = flat_classification_report(y, predict_y, labels=evaluate_labels, digits=3,output_dict=True)
    return report

In [145]:
def evaluate_model(model, target = "all"):
    TEST_MODEL = model
#     test_languages = storage.get_all_test_languages()
    test_languages = ['en','de','ru','zh','ja','ko']
    if target != "all":
        test_languages = [target]
    reports = {}
    for language in test_languages:
        print("Testing language: ", language)
        test_urls = [rec['Page URL'] for rec in storage.iter_test_records_by_language(language=language)]
        test_X_raw, test_y = storage.get_test_Xy_by_language(language=language)
        print("pages: {}  domains: {}".format(len(test_urls), len({get_domain(url) for url in test_urls})))
        _test_x, _test_y = prepare_for_testing(test_X_raw, test_y)
        report = evaluate_from_batch(TEST_MODEL, _test_x, _test_y, ['PAGE','NEXT'])
        print(pd.DataFrame(report))
        reports[language] = report
        print("===================================")
    return reports

In [147]:
def calculate_macro_avg(reports):
    avg_macro = 0
    for lan, report in reports.items():
        avg_macro+=report['macro avg']['f1-score']
    return avg_macro/len(reports)

In [327]:
test_x, test_y = storage.get_test_Xy_by_language(language='ko')
test_x, test_y = prepare_for_testing(test_x, test_y)

Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=24)

In [329]:
for inputs in test_x:
    print(inputs[0].shape)

(771, 1024)
(771, 256)
(771, 38)


### Evaluate

In [321]:
model.summary()

Model: "model_26"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_attribute (InputLayer)    [(None, None, 256)]  0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, None, 256, 69 4830        input_attribute[0][0]            
__________________________________________________________________________________________________
input_ft_embeddings (InputLayer [(None, None, 1024)] 0                                            
__________________________________________________________________________________________________
average_pooling2d_14 (AveragePo (None, None, 1, 69)  0           embedding[40][0]                 
___________________________________________________________________________________________

In [332]:
reports = evaluate_model(model)

Testing language:  en
pages: 49  domains: 34
Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=49)

Start predicting test data ...
Start evaluating test data ...
                 PAGE       NEXT   micro avg   macro avg  weighted avg
precision    0.798658   0.675676    0.774194    0.737167      0.775648
recall       0.944444   0.862069    0.929032    0.903257      0.929032
f1-score     0.865455   0.757576    0.844575    0.811515      0.845271
support    126.000000  29.000000  155.000000  155.000000    155.000000
Testing language:  de
pages: 20  domains: 7
Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=20)

/home/sam/DL/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


...

pages: 24  domains: 13
Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=24)

/home/sam/DL/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Start predicting test data ...
Start evaluating test data ...
                PAGE  NEXT  micro avg  macro avg  weighted avg
precision   0.580000   0.0   0.542056   0.290000      0.535385
recall      0.966667   0.0   0.892308   0.483333      0.892308
f1-score    0.725000   0.0   0.674419   0.362500      0.669231
support    60.000000   5.0  65.000000  65.000000     65.000000


/home/sam/DL/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['PAGE', 'NEXT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [333]:
calculate_macro_avg(reports)

0.7038363112603646

### Save best model

In [79]:
BEST_MODEL_RECORDS

[{'iter': 0, 'macro_avg': 0.6585991713693428},
 {'iter': 1, 'macro_avg': 0.7517082081030869},
 {'iter': 2, 'macro_avg': 0.7305036753424899},
 {'iter': 3, 'macro_avg': 0.6902004741461556},
 {'iter': 4, 'macro_avg': 0.7440229999359618}]

In [117]:
model.set_weights(best_model_weights)

In [116]:
checkpoint_path = "training_best/cp-best.ckpt"

In [118]:
model.save_weights(checkpoint_path)

### EMB Exp

In [205]:
EMB_EXP_RECORDS = []

In [206]:
for EMB_SIZE in [16,32,64,128]:
    for iteration in range(5):
        print("Iteration start ",iteration)
        model, loss_fn = get_custom_emb_model(use_crf=True, embedding_size = EMB_SIZE, hidden_size = 300)
        model, avg_epoch_result = train_on_epoch(25, model, optimizer, train_dataset, val_dataset)
        print("best: ",best)
        reports = evaluate_model(model)
        macro_avg = calculate_macro_avg(reports)
        print(f"Iter: {iteration}, macro f1: {macro_avg}")
        record = {"EMB_SIZE": EMB_SIZE, "iter": iteration, "macro_avg": macro_avg}
        EMB_EXP_RECORDS.append(record)

Iteration start  0

Start of epoch 0


/home/sam/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/sam/DL/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Validation loss: 34.3138
Validation F1: 0.6014
Time taken: 113.39s

Start of epoch 1
Validation loss: 17.0073
Validation F1: 0.7074
Time taken: 113.20s

Start of epoch 2
Validation loss: 18.7226
Validation F1: 0.7819
Time taken: 112.43s

Start of epoch 3
Validation loss: 14.6766
Validation F1: 0.7792
Time taken: 112.22s

Start of epoch 4
Validation loss: 12.0428
Validation F1: 0.8029
Time taken: 112.59s

Start of epoch 5
Validation loss: 11.4528
Validation F1: 0.8085
Time taken: 112.39s

Start of epoch 6
Validation loss: 16.1938
Validation F1: 0.8333
Time taken: 113.17s

Start of epoch 7
Validation loss: 11.6281
Validation F1: 0.8235
Time taken: 112.25s

Start of epoch 8
Validation loss: 12.3682
Validation F1: 0.8156
Time taken: 113.87s

Start of epoch 9
Validation loss: 19.5932
Validation F1: 0.8152
Time taken: 113.02s

Start of epoch 10
Validation loss: 15.3122
Validation F1: 0.8273
Time taken: 112.14s

Start of epoch 11
Validation loss: 13.6606
Validation F1: 0.8073
Time taken: 113.

IntProgress(value=0, description='(Init)', max=49)

/home/sam/DL/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


...

pages: 24  domains: 13
Transform key ['text-exact'] to word_vector ... 


IntProgress(value=0, description='(Init)', max=24)

/home/sam/DL/lib/python3.6/site-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


Start predicting test data ...
Start evaluating test data ...
                PAGE      NEXT  micro avg  macro avg  weighted avg
precision   0.614583  1.000000   0.618557   0.807292      0.644231
recall      0.983333  0.200000   0.923077   0.591667      0.923077
f1-score    0.756410  0.333333   0.740741   0.544872      0.723866
support    60.000000  5.000000  65.000000  65.000000     65.000000
Iter: 4, macro f1: 0.7024112646563249


/home/sam/DL/lib/python3.6/site-packages/sklearn/utils/validation.py:71: FutureWarning: Pass labels=['PAGE', 'NEXT'] as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


In [207]:
EMB_EXP_RECORDS

[{'EMB_SIZE': 16, 'iter': 0, 'macro_avg': 0.7153737382645313},
 {'EMB_SIZE': 16, 'iter': 1, 'macro_avg': 0.7409017774319923},
 {'EMB_SIZE': 16, 'iter': 2, 'macro_avg': 0.7599685070427061},
 {'EMB_SIZE': 16, 'iter': 3, 'macro_avg': 0.7020238360352109},
 {'EMB_SIZE': 16, 'iter': 4, 'macro_avg': 0.6370030763485734},
 {'EMB_SIZE': 32, 'iter': 0, 'macro_avg': 0.7439181045166969},
 {'EMB_SIZE': 32, 'iter': 1, 'macro_avg': 0.7369678480495846},
 {'EMB_SIZE': 32, 'iter': 2, 'macro_avg': 0.7757498627719578},
 {'EMB_SIZE': 32, 'iter': 3, 'macro_avg': 0.7784286026880797},
 {'EMB_SIZE': 32, 'iter': 4, 'macro_avg': 0.7849370608506332},
 {'EMB_SIZE': 64, 'iter': 0, 'macro_avg': 0.7923914176789396},
 {'EMB_SIZE': 64, 'iter': 1, 'macro_avg': 0.7566102680811095},
 {'EMB_SIZE': 64, 'iter': 2, 'macro_avg': 0.7569802491467393},
 {'EMB_SIZE': 64, 'iter': 3, 'macro_avg': 0.6934120988052667},
 {'EMB_SIZE': 64, 'iter': 4, 'macro_avg': 0.7550962051601484},
 {'EMB_SIZE': 128, 'iter': 0, 'macro_avg': 0.7389909454

In [208]:
pd.DataFrame(EMB_EXP_RECORDS)

,EMB_SIZE,iter,macro_avg
0,16,0,0.715374
1,16,1,0.740902
2,16,2,0.759969
3,16,3,0.702024
4,16,4,0.637003
5,32,0,0.743918
6,32,1,0.736968
7,32,2,0.775750
8,32,3,0.778429
9,32,4,0.784937
